In [2]:
import sys
sys.executable

'/root/home/envforir/bin/python'

In [3]:
# # 데몬 인스턴스 만들기 
import os
import json
from elasticsearch import Elasticsearch, helpers
from subprocess import Popen, PIPE, STDOUT
from dotenv import load_dotenv
import openai

# es_server = Popen(['/root/home/elasticsearch-8.8.0/bin/elasticsearch'],
#                   stdout=PIPE, 
#                   stderr=STDOUT,
#                   preexec_fn=lambda: os.setuid(1))

# import time
# time.sleep(60)

In [23]:
# 데몬 구동 확인 -> password 설정 
!ps -ef | grep elasticsearch

iruser   1102837       1 30 02:13 pts/14   00:00:52 /home/iruser/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j2.formatMsgNoLookups=true -Djava.locale.providers=SPI,COMPAT --add-opens=java.base/java.io=org.elasticsearch.preallocate -XX:+UseG1GC -Djava.io.tmpdir=/tmp/elasticsearch-15843752271474759779 -XX:+HeapDumpOnOutOfMemoryError -XX:+ExitOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,level,pid,tags:filecount=32,filesize=64m -Xms30720m -Xmx30720m -XX:MaxDirectMemorySize=16106127360 -XX:InitiatingHeapOccupancyPercent=30

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
es_username = 'elastic'
es_password = 'lIqyxP0YNWqs92cewOwj'

# ElasticSearch Client 생성 
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="/data/ephemeral/home/upstage-ai-final-ir2/HM/elasticsearch-8.8.0/config/certs/http_ca.crt", verify_certs=False)

# ElasticSearch Client 정보 확인 
es.info()

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'name': 'instance-8909', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'r4Ih9u1sTfKbhPDz61kqLQ', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [55]:
load_dotenv() 
openai_api_key = os.getenv("OPENAI_API_KEY")

In [84]:
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화(한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")   
# 이 모델을 사용하여 doc['content']를 임베딩 

# Sentence Transformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)

# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성a
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i: i+batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)    # doc의 "content" 부분을 임베딩 
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings

In [78]:
# # OpenAI를 이용하여 임베딩 생성
# def get_embeddings_in_batches(docs, batch_size=100):
#     batch_embeddings = []
#     for i in range(0, len(docs), batch_size):
#         batch = docs[i: i+batch_size]
#         contents = [doc["content"] for doc in batch]
#         embeddings = get_embedding(contents)
#         batch_embeddings.extend(embeddings)
#         print(f'batch {i} completed')
#     return batch_embeddings

# def get_embedding(sentences):
#     response = openai.Embedding.create(
#         input=sentences,
#         model="text-embedding-ada-002"
#     )
#     return [result["embedding"] for result in response["data"]]

### 인덱스 설정 함수: Sparse Retrieve, Dense Retrieve를 위한 인덱스 생성과 관리
- Dense Retrieve: 인덱스 맵핑 뿐만 아니라 밀집 벡터 필드를 추가해야

In [79]:
# 새로운 index 생성 
def create_es_index(index, settings, mappings):
    if es.indices.exists(index=index):   # 만약 인덱스가 이미 존재한다면 
        es.indices.delete(index=index)   # 새로운 것으로 설정하기 위해 삭제 
    es.indices.create(index=index, settings=settings, mappings=mappings)  # 지정된 settings로 새로운 인덱스 생성 

# 지정된 인덱스 삭제 
def delete_es_index(index):
    es.indices.delete(index=index)
    
# ElasticSearch 헬퍼 함수를 사용하여 대량 인덱싱 수행 
def bulk_add(index, docs):
    actions = [{'_index': index, '_source': doc} for doc in docs]
    return helpers.bulk(es, actions)

### 검색 함수
- Sparse Retrieve: 텍스트 기반 검색 
- Dense Retrieve: 벡터 유사도 기반 검색

In [80]:
# 역색인을 이용한 검색 
def sparse_retrieve(query_str, size):
    query = { "match": {"content": {"query": query_str}}}
    return es.search(index='test', query=query, size=size, sort='_score')
#>> query_str: 검색어
#>> size: 검색 결과 개수 
#>> 'test' 인덱스에서 정의된 쿼리를 실행 -> 검색 결과를 size개 만큼 반환 


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    query_embedding = get_embedding([query_str])[0]   # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기 
    knn = {"field": "embeddings",                     # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정 
           "query_vector": query_embedding.tolist(),
           "k": size,    # k: 검색할 최근접 이웃의 개수 
           "num_candidates": 100}  # KNN 검색에 사용할 후보 문서 개수 
    return es.search(index='test', knn=knn)           # 지정된 인덱스에서 벡터 유사도 검색 수행 

### 설정

In [81]:
# 색인을 위한 settings 설정 
settings = { "analysis": {"analyzer": {"nori": {"type": "custom",  #>> custom: 사용자 정의 분석기임을 명시 
                                                "tokenizer": "nori_tokenizer",
                                                "decompound_mode": "mixed",  # 복합 명사를 분해하는 방식을 설정. mixed: 복합명사를 분해하되, 분해된 토큰과 원본 토큰을 모두 저장 
                                                "filter": ["nori_posfilter"]}},
                          "filter": {"nori_posfilter":{"type": "nori_part_of_speech",
                                                       "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]}}}  # 제거할 품사 태그를 지정 
            }

# 색인을 위한 mapping 설정(역색인 필드, 임베딩 필드 모두 설정)
mappings = { "properties": {"content": {"type": "text",
                                        "analyzer": "nori"},
                            "embeddings": {"type": "dense_vector",
                                           "dims": 768,
                                           "index": True,
                                           "similarity": "l2_norm"}}  # 벡터 간 유사도 계산 방식: L2 norm 
            }


In [85]:
# 위에서 설정한 내용으로 'test' 인덱스 생성 
create_es_index("test", settings, mappings)

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### 임베딩: 문서의 content 필드

In [86]:
# 문서의 content 필드에 대한 임베딩 생성
index_docs = []
with open("../data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)
                
# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding.tolist()
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)

batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200


/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/c

(4272, [])


In [87]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query = "각 나라에서의 공교육 지출 현황에 대해 알려줘."

### Sparse Retrieval 예제 

In [88]:
# 역색인을 사용하는 검색 예제 
search_result_retrieve = sparse_retrieve(test_query, 10)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']['content'])

score: 17.829744 source: 2017년 현재, 전세계의 공공 교육 지출은 세계 GDP의 약 4%를 차지하고 있습니다. 이는 국가들이 교육에 많은 자원을 투자하고 있다는 것을 보여줍니다. 공공 교육은 사회의 발전과 경제 성장에 중요한 역할을 합니다. 국가들은 교육을 통해 인재를 양성하고, 인구의 교육 수준을 높여 경제적인 경쟁력을 강화하고자 합니다. 따라서 많은 국가들이 교육에 많은 예산을 할당하고 있으며, 이는 세계 GDP의 4%에 해당하는 규모입니다. 이러한 투자는 국가의 교육 체계를 강화하고, 국민들의 교육 기회를 확대하는 데 도움을 줍니다. 공공 교육 지출은 국가의 교육 정책과 우선순위를 반영하며, 국가의 교육 수준과 경제 성장에 직결되는 중요한 지표입니다.
score: 13.655198 source: 세계 의료비 총지출은 2017년 현재 세계 GDP의 약 10%를 차지하고 있습니다. 이는 세계 각국이 건강과 의료에 많은 자원을 투자하고 있다는 것을 보여줍니다. 의료비 총지출은 의료 시스템의 발전과 인구의 건강 수준 향상을 위해 중요한 지표로 사용됩니다. 세계 의료비 총지출이 GDP의 10%에 이르는 것은 의료 분야에 대한 긍정적인 투자와 의료 기술의 발전을 반영하는 결과입니다. 이러한 투자는 인류의 건강과 복지를 증진시키는 데 큰 역할을 합니다.
score: 12.250196 source: 전기력은 전하 사이에 작용하는 힘입니다. 이 문제에서는 두 개의 전기를 띠는 공이 짧은 거리로 분리되어 있고, 그 사이에 50μN의 힘이 발생한다고 주어졌습니다. 각 공의 전하는 동일하게 유지되면서, 한 공의 질량만 두 배로 증가시킵니다.

전기력은 전하의 크기와 거리에 비례하며, 전하의 크기와 질량은 독립적인 개념입니다. 따라서 한 공의 질량이 두 배로 증가하더라도, 다른 공의 전하는 변하지 않습니다.

따라서, 공 사이의 새로운 전기 힘은 여전히 50μN입니다.
score: 12.002875 source: 인도는 현재 정당 부패에 대한 의식이 높은

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [65]:
# dense_retrieve_2: sparse_retrieve 함수로 필터링된 문서들에 대해서만 KNN 검색
def dense_retrieve_2(query_str, documents, size):
    query_embedding = get_embedding([query_str][0])
    document_ids = [doc['_id'] for doc in documents]
    # document_embeddings = [doc['_source']['embeddings'] for doc in documents]
    
    knn_query = {
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": len(documents)
    }
    
    # 미리 필터링된 문서들에 대해서만 'test' 인덱스에서 KNN 검색 수행 
    # KNN 검색을 위해 적절한 body 구조 생성
    body = {
        "_source": {"includes": ["content", "embeddings"]},
        "query": {
            "bool": {
                "filter": [
                    {"terms": {"_id": document_ids}}
                ]
            }
        },
        "knn": knn_query
    }
    
    # 실제로 반환된 결과의 개수를 제한하기 위해 size 파라미터를 사용
    response = es.search(index='test', body=body, size=size)
    
    return es.search(index='test', body=body)
    

### dense_retrieve_2 예제 

In [66]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve_2(test_query, search_result_retrieve['hits']['hits'], 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.0032792056 source: 여러 미개발국가들의 중등학교 등록 수준은 각기 다릅니다. 그 중에서도 2020년 현재 중등학교 등록률이 높은 국가들로는 콜롬비아, 페루, 우루과이, 코스타리카가 있습니다. 이들 국가들은 교육에 많은 투자를 하고 있으며, 학생들의 교육 기회를 확대하기 위해 노력하고 있습니다. 콜롬비아는 높은 등록률을 유지하면서도 교육의 질을 높이기 위해 교사들의 교육을 강화하고 있습니다. 페루는 국가 전체적으로 중등학교 등록률을 높이기 위해 정책적인 지원을 하고 있으며, 우루과이는 교육 시스템을 혁신하여 학생들의 학습 효과를 극대화하고 있습니다. 코스타리카는 교육에 대한 사회적인 인식을 높이고, 학생들의 학습 환경을 개선하기 위해 다양한 프로그램을 운영하고 있습니다. 이러한 노력들이 중등학교 등록률을 높이는 데에 큰 역할을 하고 있습니다.
score: 0.0030429724 source: 에티오피아에서는 2009년에 8세 아동들 중 38%가 지난 주에 선생님으로부터 체벌을 받았다고 보고했습니다. 이는 매우 높은 비율입니다. 아동들은 학교에서 안전하게 배울 수 있어야 합니다. 그러나 이러한 체벌은 아동들의 신체적, 정서적, 인지적 발달에 부정적인 영향을 미칠 수 있습니다. 따라서 에티오피아 교육 당국은 이 문제에 대해 심각하게 대처해야 합니다. 체벌을 받은 아동들을 보호하고, 교사들에게 교육적인 대안을 제공하는 것이 필요합니다. 이를 통해 에티오피아의 교육 환경을 개선하고, 모든 아동들이 안전하게 학교에서 배울 수 있는 기회를 가질 수 있도록 해야 합니다.
score: 0.0029874237 source: 2014년 현재, 사하라 이남 아프리카 국가들 중 초등학교 재학 연령임에도 학교를 다니지 않는 아동은 약 20%입니다. 이는 매우 높은 비율로, 아프리카 대륙에서 교육의 어려움을 보여주는 지표입니다. 이러한 상황은 경제적인 이유, 가족의 사회적인 압력, 교육 시설의 부족 등 다양한 요인으로 인해 발생할 수 있습니다. 이러한 문제

/tmp/ipykernel_1102401/2457609031.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index='test', body=body, size=size)
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/tmp/ipykernel_1102401/2457609031.py:31: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index='test', body=body)
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.

### Dense Retrieval 예제

In [67]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.0032792056 source: 여러 미개발국가들의 중등학교 등록 수준은 각기 다릅니다. 그 중에서도 2020년 현재 중등학교 등록률이 높은 국가들로는 콜롬비아, 페루, 우루과이, 코스타리카가 있습니다. 이들 국가들은 교육에 많은 투자를 하고 있으며, 학생들의 교육 기회를 확대하기 위해 노력하고 있습니다. 콜롬비아는 높은 등록률을 유지하면서도 교육의 질을 높이기 위해 교사들의 교육을 강화하고 있습니다. 페루는 국가 전체적으로 중등학교 등록률을 높이기 위해 정책적인 지원을 하고 있으며, 우루과이는 교육 시스템을 혁신하여 학생들의 학습 효과를 극대화하고 있습니다. 코스타리카는 교육에 대한 사회적인 인식을 높이고, 학생들의 학습 환경을 개선하기 위해 다양한 프로그램을 운영하고 있습니다. 이러한 노력들이 중등학교 등록률을 높이는 데에 큰 역할을 하고 있습니다.
score: 0.0030429724 source: 에티오피아에서는 2009년에 8세 아동들 중 38%가 지난 주에 선생님으로부터 체벌을 받았다고 보고했습니다. 이는 매우 높은 비율입니다. 아동들은 학교에서 안전하게 배울 수 있어야 합니다. 그러나 이러한 체벌은 아동들의 신체적, 정서적, 인지적 발달에 부정적인 영향을 미칠 수 있습니다. 따라서 에티오피아 교육 당국은 이 문제에 대해 심각하게 대처해야 합니다. 체벌을 받은 아동들을 보호하고, 교사들에게 교육적인 대안을 제공하는 것이 필요합니다. 이를 통해 에티오피아의 교육 환경을 개선하고, 모든 아동들이 안전하게 학교에서 배울 수 있는 기회를 가질 수 있도록 해야 합니다.
score: 0.0029874237 source: 2014년 현재, 사하라 이남 아프리카 국가들 중 초등학교 재학 연령임에도 학교를 다니지 않는 아동은 약 20%입니다. 이는 매우 높은 비율로, 아프리카 대륙에서 교육의 어려움을 보여주는 지표입니다. 이러한 상황은 경제적인 이유, 가족의 사회적인 압력, 교육 시설의 부족 등 다양한 요인으로 인해 발생할 수 있습니다. 이러한 문제

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# RAG 구현
준비된 검색엔진과 LLM을 활용하셔 대화형 RAG 구현

In [93]:
import os
import openai
import traceback

# OpenAI 클라이언트 초기화
openai.api_key = os.getenv("OPENAI_API_KEY")

# 사용할 모델 설정 (여기서는 gpt-3.5-turbo 모델 사용)
llm_model = "gpt-3.5-turbo"

In [94]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "Final query suitable for use in search from the user messages history."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


In [98]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = openai.ChatCompletion.create(
            model=llm_model,
            messages=msg,
            temperature=0,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    try:
        function_args = result.choices[0].message.get('function_call', {}).get('arguments', None)
        if function_args:
            function_args = json.loads(function_args)
            standalone_query = function_args.get("standalone_query")

            # retriever 설정!!!
            search_result = dense_retrieve(standalone_query, 3)

            response["standalone_query"] = standalone_query
            retrieved_context = []
            for i, rst in enumerate(search_result['hits']['hits']):
                retrieved_context.append(rst["_source"]["content"])
                response["topk"].append(rst["_source"]["docid"])
                response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

            content = json.dumps(retrieved_context)
            messages.append({"role": "assistant", "content": content})
            msg = [{"role": "system", "content": persona_qa}] + messages
            try:
                qaresult = openai.ChatCompletion.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    timeout=30
                )
            except Exception as e:
                traceback.print_exc()
                return response
            response["answer"] = qaresult.choices[0].message['content']
        else:
            response["answer"] = result.choices[0].message['content']
    except AttributeError as e:
        traceback.print_exc()
        response["answer"] = result.choices[0].message['content']

    return response



In [99]:
# def answer_question(messages):
#     # 함수 출력 초기화
#     response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

#     # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
#     msg = [{"role": "system", "content": persona_function_calling}] + messages
#     try:
#         result = client.chat.completions.create(
#             model=llm_model,
#             messages=msg,
#             tools=tools,
#             temperature=0,
#             seed=1,
#             timeout=10
#         )
#     except Exception as e:
#         traceback.print_exc()
#         return response

#     # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
#     if result.choices[0].message.tool_calls:
#         tool_call = result.choices[0].message.tool_calls[0]
#         function_args = json.loads(tool_call.function.arguments)
#         standalone_query = function_args.get("standalone_query")

#         # Baseline으로 sparse_retrieve로 10개 결과 추출
#         initial_search_result = sparse_retrieve(standalone_query, 10)

#         # sparse_retrieve 결과를 dense_retrieve_2로 3개 결과 추출
#         filtered_search_result = dense_retrieve_2(standalone_query, initial_search_result['hits']['hits'], 3)

#         # response["standalone_query"] = standalone_query
#         # retrieved_context = []
#         # for i, rst in enumerate(filtered_search_result['hits']['hits']):
#         #     retrieved_context.append(rst["_source"]["content"])
#         #     response["topk"].append(rst["_source"]["docid"])
#         #     response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

#         response["standalone_query"] = standalone_query
#         retrieved_context = []
#         for i, rst in enumerate(filtered_search_result['hits']['hits']):
#             retrieved_context.append(rst["_source"]["content"])
#             response["topk"].append(rst["_id"])  # 여기를 수정: "docid" 대신 "_id" 사용
#             response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})
        
#         content = json.dumps(retrieved_context)
#         messages.append({"role": "assistant", "content": content})
#         msg = [{"role": "system", "content": persona_qa}] + messages
#         try:
#             qaresult = client.chat.completions.create(
#                 model=llm_model,
#                 messages=msg,
#                 temperature=0,
#                 seed=1,
#                 timeout=30
#             )
#         except Exception as e:
#             traceback.print_exc()
#             return response
#         response["answer"] = qaresult.choices[0].message.content

#     # 검색이 필요하지 않은 경우 바로 답변 생성
#     else:
#         response["answer"] = result.choices[0].message.content

#     return response


In [100]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx > 5:
            #   break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [101]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("/data/ephemeral/home/upstage-ai-final-ir2/HM/data/eval.jsonl", "../output/submission_12.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Answer: 나무의 분류를 조사하기 위한 방법은 주로 식물학적인 방법을 사용합니다. 일반적으로 다음과 같은 단계를 따릅니다:
1. 잎, 꽃, 열매 등의 외부 형태를 관찰하여 식별합니다.
2. 나무의 잎의 형태, 잎의 배치, 꽃과 열매의 형태 등을 토대로 식별합니다.
3. 꽃의 구조, 열매의 특징, 줄기의 형태 등을 통해 나무를 분류합니다.
4. 분류된 나무를 식물 분류표를 참고하여 명확히 식별합니다.

이외에도 DNA 분석을 통해 나무를 분류하는 방법도 있습니다. DNA 분석을 통해 유전적으로 유사한 나무들을 그룹화하여 분류할 수 있습니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 죄송합니다, 제가 직접 정보를 제공해 드릴 수는 없지만, 인터넷을 통해 각 나라에서의 공교육 지출 현황에 대한 자세한 정보를 얻을 수 있습니다. 국제기구나 각 나라 정부의 통계청 웹사이트에서 해당 정보를 확인할 수 있습니다. 혹시 다른 과학 관련 질문이 있으신가요?

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer: 기억 상실증은 다양한 원인으로 발생할 수 있습니다. 뇌 손상, 뇌졸중, 알츠하이머병, 외상, 스트레스, 우울증, 뇌졸중 등이 그 중 일부 원인으로 알려져 있습니다. 정확한 진단과 치료를 위해서는 전문가의 상담이 필요합니다.

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
Answer: 통학

KeyboardInterrupt: 